# Enhanced ATM Straddle Strategy - Using Backtester Framework

This notebook demonstrates the enhanced ATM straddle strategy using the built-in backtesting framework.

## Strategy Overview

- **Strategy Type**: Short ATM Straddle (sell volatility)
- **Underlying**: SPY
- **Entry**: Daily (when conditions met)
- **DTE Target**: 11-18 days
- **Exit Rules**:
  - Profit Target: 25% of premium received
  - Stop Loss: 100% of premium (2x max loss)
  - Time Stop: DTE = 1 (avoid gamma risk)

## Framework Components Used

This notebook properly uses the backtester framework:
- `MarketData` and `MarketDataLoader` for data management
- `BacktestEngine` and `BacktestConfig` for orchestration
- `StraddleStrategy` for position creation
- `BlackScholesModel` for pricing
- `PerformanceMetrics` for analytics
- `VisualizationEngine` for charts

## 1. Setup and Imports

In [ ]:
import sys
from pathlib import Path

# Add parent directory to path to import backtester module
parent_dir = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Import backtester modules
from backtester import (
    MarketData,
    MarketDataLoader,
    DoltHubAdapter,
    BacktestEngine,
    BacktestConfig,
    StraddleStrategy,
    OptionContract,
    Portfolio,
    BlackScholesModel,
    PerformanceMetrics,
    VisualizationEngine
)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Plot settings
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("Setup complete!")
print(f"Python path includes: {parent_dir}")

## 2. Configuration

In [ ]:
# Strategy configuration
TICKER = 'SPY'
START_DATE = '2022-01-03'
END_DATE = '2024-12-31'
INITIAL_CAPITAL = 100000.0

# DTE parameters
MIN_DTE = 11
MAX_DTE = 18
TARGET_DTE = 14

# Exit parameters
PROFIT_TARGET_PCT = 0.25  # 25% of premium received
STOP_LOSS_PCT = 1.00      # 100% loss (2x breakeven)
TIME_STOP_DTE = 1         # Exit at DTE=1

# Position sizing
CONTRACTS_PER_TRADE = 1

# Data source
DOLT_DB_PATH = '/Users/janussuk/Desktop/dolt_data/options'

print(f"Strategy: Short ATM Straddle on {TICKER}")
print(f"Period: {START_DATE} to {END_DATE}")
print(f"DTE Range: {MIN_DTE}-{MAX_DTE} days")
print(f"Initial Capital: ${INITIAL_CAPITAL:,.0f}")

## 3. Load Market Data

Use the framework's `MarketDataLoader` and `DoltHubAdapter` to load data.

In [ ]:
# Initialize adapter and loader (framework components)
adapter = DoltHubAdapter(database_path=DOLT_DB_PATH)
loader = MarketDataLoader(adapter)

# Load market data with vol surfaces (framework method)
market_data = loader.load(
    ticker=TICKER,
    start_date=START_DATE,
    end_date=END_DATE,
    build_vol_surface=True
)

print(f"\nMarket data loaded successfully!")
print(f"Date range: {market_data.time_index[0]} to {market_data.time_index[-1]}")
print(f"Total days: {len(market_data.time_index)}")
print(f"Volatility surfaces: {len(market_data.vol_surfaces)}")

## 4. Load Options Data

Load the full options chain for strategy selection.

In [ ]:
# Load options data using adapter (framework method)
options_df = adapter.load_option_data(
    ticker=TICKER,
    start_date=START_DATE,
    end_date=END_DATE
)

# Calculate DTE for each option
options_df['dte'] = (options_df['expiration'] - options_df['date']).dt.days

# Filter to target DTE range
options_df = options_df[
    (options_df['dte'] >= MIN_DTE) & 
    (options_df['dte'] <= MAX_DTE)
].copy()

print(f"\nOptions data loaded!")
print(f"Total option records: {len(options_df):,}")
print(f"Date range: {options_df['date'].min()} to {options_df['date'].max()}")
print(f"DTE range: {options_df['dte'].min()} to {options_df['dte'].max()}")
print(f"\nSample data:")
print(options_df.head())

## 5. Extended BacktestEngine with Profit Target/Stop Loss

Extend the framework's `BacktestEngine` to add profit target and stop loss functionality while using all framework components.

In [ ]:
class StraddleBacktestEngine(BacktestEngine):
    """
    Extended BacktestEngine with profit target and stop loss exits.
    Uses framework's StraddleStrategy and all other components.
    """
    
    def __init__(
        self,
        market_data: MarketData,
        config: BacktestConfig,
        options_df: pd.DataFrame,
        ticker: str,
        target_dte: int = 14,
        profit_target_pct: float = 0.25,
        stop_loss_pct: float = 1.00,
        time_stop_dte: int = 1
    ):
        super().__init__(market_data, config)
        self.options_df = options_df
        self.ticker = ticker
        self.target_dte = target_dte
        self.profit_target_pct = profit_target_pct
        self.stop_loss_pct = stop_loss_pct
        self.time_stop_dte = time_stop_dte
        
        # Track strategy entry values for P&L calculation
        self.strategy_entry_values = {}  # strategy_index -> entry_value
        
    def find_atm_straddle_params(self, date: pd.Timestamp, spot: float):
        """
        Find ATM straddle parameters for a given date.
        Returns strike and expiry for creating a StraddleStrategy.
        """
        # Filter to this date
        daily_options = self.options_df[self.options_df['date'] == date].copy()
        
        if daily_options.empty:
            return None
        
        # Find expiration closest to target DTE
        expirations = daily_options.groupby('expiration')['dte'].first().reset_index()
        expirations['dte_diff'] = abs(expirations['dte'] - self.target_dte)
        best_expiry = expirations.sort_values('dte_diff').iloc[0]['expiration']
        
        # Filter to this expiration
        expiry_options = daily_options[daily_options['expiration'] == best_expiry].copy()
        
        # Find ATM strike (closest to spot)
        strikes = expiry_options['strike'].unique()
        atm_strike = strikes[np.argmin(np.abs(strikes - spot))]
        
        return {
            'strike': float(atm_strike),
            'expiry': best_expiry,
            'dte': int(expirations.loc[expirations['expiration'] == best_expiry, 'dte'].iloc[0])
        }
    
    def run(self) -> pd.DataFrame:
        """
        Run the backtest with daily entry and exit logic.
        """
        # Get trading dates
        trading_dates = self.market_data.time_index[
            (self.market_data.time_index >= self.config.start_date) &
            (self.market_data.time_index <= self.config.end_date)
        ]

        print(f"Running backtest from {self.config.start_date} to {self.config.end_date}")
        print(f"Trading days: {len(trading_dates)}\n")

        for i, date in enumerate(trading_dates):
            if (i + 1) % 50 == 0:
                print(f"Processing day {i+1}/{len(trading_dates)}...")
            
            self.current_date = date
            self._process_day_with_exits(date)

        # Convert results to DataFrame
        results_df = pd.DataFrame(self.daily_results)
        if not results_df.empty:
            results_df.set_index('date', inplace=True)

        print(f"\nBacktest complete!")
        print(f"Final portfolio value: ${results_df['portfolio_value'].iloc[-1]:,.2f}")
        print(f"Total return: {results_df['total_return'].iloc[-1]:.2%}")
        print(f"Total trades: {len(self.portfolio.trade_history)}") 

        return results_df
    
    def _process_day_with_exits(self, date: pd.Timestamp):
        """
        Process a single trading day with exit logic.
        """
        # 1. Get market state
        spot = self.market_data.get_spot(date)
        
        # 2. Check for exits (profit target, stop loss, time stop)
        self._check_and_exit_positions(date, spot)
        
        # 3. Enter new position if no open positions
        if len(self.portfolio.strategies) == 0:
            self._enter_new_straddle(date, spot)
        
        # 4. Calculate portfolio value and Greeks
        portfolio_value = self.portfolio.value(
            date, self.market_data, self.config.model
        )
        portfolio_greeks = self.portfolio.greeks(
            date, self.market_data, self.config.model
        )
        
        # 5. Calculate P&L
        daily_pnl = portfolio_value - self.prev_portfolio_value
        total_return = (portfolio_value - self.config.initial_capital) / self.config.initial_capital
        
        # 6. P&L Attribution (if not first day)
        if self.prev_portfolio_value != self.config.initial_capital:
            pnl_attribution = self._calculate_pnl_attribution(
                date, spot, portfolio_greeks
            )
        else:
            pnl_attribution = {
                'pnl_delta': 0.0,
                'pnl_gamma': 0.0,
                'pnl_vega': 0.0,
                'pnl_theta': 0.0,
                'pnl_rho': 0.0,
                'pnl_residual': 0.0
            }
        
        # 7. Handle expirations
        self._handle_expirations(date)
        
        # 8. Record daily results
        result = {
            'date': date,
            'spot': spot,
            'portfolio_value': portfolio_value,
            'cash': self.portfolio.cash,
            'daily_pnl': daily_pnl,
            'total_return': total_return,
            **portfolio_greeks,
            **pnl_attribution,
            'num_strategies': len(self.portfolio.strategies),
        }
        
        self.daily_results.append(result)
        
        # 9. Update state for next day
        self.prev_portfolio_value = portfolio_value
        self.prev_greeks = portfolio_greeks.copy()
    
    def _enter_new_straddle(self, date: pd.Timestamp, spot: float):
        """
        Enter new short straddle using framework's StraddleStrategy.
        """
        params = self.find_atm_straddle_params(date, spot)
        
        if params is None:
            return
        
        # Create StraddleStrategy using framework
        strategy = StraddleStrategy(
            underlying=self.ticker,
            strike=params['strike'],
            expiry=params['expiry'],
            direction='short',  # We're selling the straddle
            quantity=1.0
        )
        
        # Calculate entry value
        entry_value = strategy.value(date, self.market_data, self.config.model)
        
        # Add strategy to portfolio using framework method
        self.portfolio.add_strategy(strategy)
        
        # Calculate transaction costs
        num_contracts = sum(abs(leg.quantity) for leg in strategy.legs)
        txn_cost = (
            num_contracts * self.config.transaction_cost_per_contract +
            abs(entry_value) * self.config.transaction_cost_pct
        )
        
        total_cost = entry_value + txn_cost
        
        # Record trade using framework method
        self.portfolio.record_trade(
            date=date,
            description=f"Enter Short Straddle @ {params['strike']:.2f}, DTE={params['dte']}",
            cash_flow=-total_cost,
            strategy_index=len(self.portfolio.strategies) - 1
        )
        
        # Store entry value for exit P&L calculation
        strategy_index = len(self.portfolio.strategies) - 1
        self.strategy_entry_values[strategy_index] = {
            'entry_value': entry_value,
            'entry_date': date,
            'entry_dte': params['dte'],
            'strike': params['strike'],
            'expiry': params['expiry']
        }
    
    def _check_and_exit_positions(self, date: pd.Timestamp, spot: float):
        """
        Check all open positions for exit conditions.
        """
        strategies_to_remove = []
        
        for i, strategy in enumerate(self.portfolio.strategies):
            # Get strategy entry info
            if i not in self.strategy_entry_values:
                continue
            
            entry_info = self.strategy_entry_values[i]
            
            # Calculate current value
            current_value = strategy.value(date, self.market_data, self.config.model)
            
            # P&L (for short position, we want value to decrease)
            pnl = entry_info['entry_value'] - current_value
            
            # Calculate DTE
            dte = (entry_info['expiry'] - date).days
            
            # Check exit conditions
            should_exit = False
            exit_reason = None
            
            # Time stop
            if dte <= self.time_stop_dte:
                should_exit = True
                exit_reason = 'time_stop'
            
            # Profit target (P&L is positive and >= target)
            elif pnl >= abs(entry_info['entry_value']) * self.profit_target_pct:
                should_exit = True
                exit_reason = 'profit_target'
            
            # Stop loss (P&L is negative and <= stop)
            elif pnl <= -abs(entry_info['entry_value']) * self.stop_loss_pct:
                should_exit = True
                exit_reason = 'stop_loss'
            
            if should_exit:
                # Calculate exit transaction costs
                num_contracts = sum(abs(leg.quantity) for leg in strategy.legs)
                txn_cost = (
                    num_contracts * self.config.transaction_cost_per_contract +
                    abs(current_value) * self.config.transaction_cost_pct
                )
                
                # Record exit trade
                self.portfolio.record_trade(
                    date=date,
                    description=f"Exit {strategy.name} - {exit_reason} (P&L: ${pnl:.2f}, DTE: {dte})",
                    cash_flow=current_value - txn_cost,
                    strategy_index=i
                )
                
                strategies_to_remove.append(i)
        
        # Remove exited strategies
        for i in sorted(strategies_to_remove, reverse=True):
            self.portfolio.remove_strategy(i)
            del self.strategy_entry_values[i]
            
            # Re-index remaining strategies
            new_entry_values = {}
            for key, value in self.strategy_entry_values.items():
                new_key = key if key < i else key - 1
                new_entry_values[new_key] = value
            self.strategy_entry_values = new_entry_values

print("Extended BacktestEngine defined!")

## 6. Run Backtest

Create configuration and run the backtest using the extended engine.

In [ ]:
# Create BacktestConfig using framework
config = BacktestConfig(
    start_date=pd.Timestamp(START_DATE),
    end_date=pd.Timestamp(END_DATE),
    initial_capital=INITIAL_CAPITAL,
    transaction_cost_per_contract=1.00,
    transaction_cost_pct=0.0005,  # 0.05%
    model=BlackScholesModel(use_market_iv=True)
)

# Create extended backtest engine
backtest = StraddleBacktestEngine(
    market_data=market_data,
    config=config,
    options_df=options_df,
    ticker=TICKER,
    target_dte=TARGET_DTE,
    profit_target_pct=PROFIT_TARGET_PCT,
    stop_loss_pct=STOP_LOSS_PCT,
    time_stop_dte=TIME_STOP_DTE
)

# Run backtest
results_df = backtest.run()

print(f"\nBacktest complete!")
print(f"Results shape: {results_df.shape}")

## 7. Get Trade History

Extract trades from the framework's Portfolio trade history.

In [ ]:
# Get trade history from portfolio (framework method)
trades_df = backtest.portfolio.get_trade_history()

if trades_df.empty:
    print("No trades executed during backtest period.")
    # Create empty dataframes for downstream cells
    entry_trades = pd.DataFrame()
    exit_trades = pd.DataFrame()
else:
    # Parse trade descriptions to extract exit reasons and other info
    def parse_trade_description(desc):
        """Extract info from trade description."""
        if 'Enter' in desc:
            return 'entry', None, None
        elif 'Exit' in desc:
            # Extract exit reason
            if 'profit_target' in desc:
                reason = 'profit_target'
            elif 'stop_loss' in desc:
                reason = 'stop_loss'
            elif 'time_stop' in desc:
                reason = 'time_stop'
            else:
                reason = 'other'
            
            # Extract P&L and DTE if available
            pnl = None
            dte = None
            if 'P&L:' in desc:
                try:
                    pnl_str = desc.split('P&L: $')[1].split(',')[0]
                    pnl = float(pnl_str)
                except:
                    pass
            if 'DTE:' in desc:
                try:
                    dte_str = desc.split('DTE: ')[1].split(')')[0]
                    dte = int(dte_str)
                except:
                    pass
            
            return 'exit', reason, {'pnl': pnl, 'dte': dte}
        else:
            return 'other', None, None

    trades_df['trade_type'], trades_df['exit_reason'], trades_df['extra_info'] = zip(
        *trades_df['description'].apply(parse_trade_description)
    )

    # Separate entry and exit trades
    entry_trades = trades_df[trades_df['trade_type'] == 'entry'].copy()
    exit_trades = trades_df[trades_df['trade_type'] == 'exit'].copy()

    print(f"Total trades: {len(trades_df)}")
    print(f"Entry trades: {len(entry_trades)}")
    print(f"Exit trades: {len(exit_trades)}")
    
    if not exit_trades.empty:
        print(f"\nExit reason breakdown:")
        print(exit_trades['exit_reason'].value_counts())
    
    print(f"\nRecent trades:")
    print(trades_df[['date', 'description', 'cash_flow']].tail(10))

## 8. Performance Metrics Using Framework

Use the framework's PerformanceMetrics class for analysis.

In [ ]:
# Create PerformanceMetrics instance (framework class)
metrics = PerformanceMetrics(results_df, risk_free_rate=0.05)

# Calculate metrics using framework methods
total_return = metrics.total_return()
annualized_return = metrics.annualized_return()
volatility = metrics.annualized_volatility()
sharpe = metrics.sharpe_ratio()
sortino = metrics.sortino_ratio()
max_dd, peak_date, trough_date = metrics.max_drawdown()  # Returns tuple
calmar = metrics.calmar_ratio()

# Calculate additional trade statistics
if not exit_trades.empty:
    exit_pnls = exit_trades['extra_info'].apply(lambda x: x['pnl'] if x and x['pnl'] is not None else 0)
    winning_exits = exit_pnls[exit_pnls > 0]
    losing_exits = exit_pnls[exit_pnls <= 0]

    win_rate = len(winning_exits) / len(exit_pnls) if len(exit_pnls) > 0 else 0
    avg_win = winning_exits.mean() if len(winning_exits) > 0 else 0
    avg_loss = losing_exits.mean() if len(losing_exits) > 0 else 0
    profit_factor = abs(winning_exits.sum() / losing_exits.sum()) if losing_exits.sum() != 0 else float('inf')
else:
    exit_pnls = pd.Series()
    win_rate = 0
    avg_win = 0
    avg_loss = 0
    profit_factor = 0

print("=" * 60)
print("PERFORMANCE METRICS (Framework PerformanceMetrics)")
print("=" * 60)
print(f"Total Return: {total_return:.2%}")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {volatility:.2%}")
print(f"Sharpe Ratio: {sharpe:.3f}")
print(f"Sortino Ratio: {sortino:.3f}")
print(f"Maximum Drawdown: {max_dd:.2%}")
print(f"Calmar Ratio: {calmar:.3f}")
print(f"")
print(f"Total Trades: {len(exit_trades)}")
print(f"Win Rate: {win_rate:.1%}")
print(f"Profit Factor: {profit_factor:.2f}")
print(f"Average Win: ${avg_win:,.2f}")
print(f"Average Loss: ${avg_loss:,.2f}")
print("=" * 60)

## 9. Visualizations Using Framework

Use the framework's VisualizationEngine for charts.

In [ ]:
# Create VisualizationEngine instance (framework class)
viz = VisualizationEngine(use_plotly=False)  # Use matplotlib for notebook

# 1. Equity Curve (framework method)
viz.plot_equity_curve(results_df, title="Short ATM Straddle - Equity Curve")

In [ ]:
# 2. Drawdown Chart (framework method)
viz.plot_drawdown(results_df, title="Portfolio Drawdown")

In [ ]:
# 3. Returns Distribution (framework method)
viz.plot_returns_distribution(results_df, title="Daily Returns Distribution")

In [ ]:
# 4. Greeks Over Time (framework method)
viz.plot_greeks(results_df, title="Portfolio Greeks Over Time")

## 10. Custom Analysis Charts

In [ ]:
# Additional custom visualizations
if exit_trades.empty or trades_df.empty:
    print("No trades to visualize. Skipping custom analysis charts.")
else:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Strategy Performance Analysis', fontsize=16, fontweight='bold')

    # 1. Exit Reason Pie Chart
    ax1 = axes[0, 0]
    exit_counts = exit_trades['exit_reason'].value_counts()
    colors_pie = {'profit_target': 'green', 'stop_loss': 'red', 'time_stop': 'orange'}
    pie_colors = [colors_pie.get(reason, 'gray') for reason in exit_counts.index]
    ax1.pie(exit_counts.values, labels=exit_counts.index, autopct='%1.1f%%', 
            startangle=90, colors=pie_colors)
    ax1.set_title('Exit Reason Distribution', fontweight='bold')

    # 2. Cumulative P&L
    ax2 = axes[0, 1]
    cumulative_pnl = trades_df['cash_flow'].cumsum()
    ax2.plot(range(len(cumulative_pnl)), cumulative_pnl, linewidth=2, color='darkblue')
    ax2.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    ax2.set_title('Cumulative Cash Flow', fontweight='bold')
    ax2.set_xlabel('Trade Number')
    ax2.set_ylabel('Cumulative Cash Flow ($)')
    ax2.grid(True, alpha=0.3)

    # 3. Trade P&L Distribution
    ax3 = axes[1, 0]
    valid_pnls = exit_pnls[exit_pnls != 0]
    if len(valid_pnls) > 0:
        ax3.hist(valid_pnls, bins=min(30, len(valid_pnls)), color='steelblue', alpha=0.7, edgecolor='black')
        ax3.axvline(x=0, color='red', linestyle='--', linewidth=2)
    ax3.set_title('Exit P&L Distribution', fontweight='bold')
    ax3.set_xlabel('P&L ($)')
    ax3.set_ylabel('Frequency')
    ax3.grid(True, alpha=0.3, axis='y')

    # 4. Equity vs Benchmark
    ax4 = axes[1, 1]
    ax4.plot(results_df.index, results_df['portfolio_value'], 
             linewidth=2, color='navy', label='Strategy')
    ax4.plot(results_df.index, results_df['spot'] / results_df['spot'].iloc[0] * INITIAL_CAPITAL,
             linewidth=2, color='gray', alpha=0.7, label=f'{TICKER} Buy & Hold')
    ax4.set_title('Strategy vs Buy & Hold', fontweight='bold')
    ax4.set_xlabel('Date')
    ax4.set_ylabel('Portfolio Value ($)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    ax4.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

    plt.tight_layout()
    plt.show()

## 11. Summary Statistics Table

In [ ]:
# Create comprehensive summary using framework metrics
summary = pd.DataFrame({
    'Metric': [
        'Initial Capital',
        'Final Equity',
        'Total Return',
        'Annualized Return',
        'Volatility (Ann.)',
        'Sharpe Ratio',
        'Sortino Ratio',
        'Max Drawdown',
        'Calmar Ratio',
        '',
        'Total Trades',
        'Win Rate',
        'Profit Factor',
        'Avg Win',
        'Avg Loss',
        '',
        'Best Trade',
        'Worst Trade',
        'Total Cash Flow'
    ],
    'Value': [
        f'${INITIAL_CAPITAL:,.0f}',
        f'${results_df["portfolio_value"].iloc[-1]:,.2f}',
        f'{total_return:.2%}',
        f'{annualized_return:.2%}',
        f'{volatility:.2%}',
        f'{sharpe:.3f}',
        f'{sortino:.3f}',
        f'{max_dd:.2%}',
        f'{calmar:.3f}',
        '',
        f'{len(exit_trades)}',
        f'{win_rate:.1%}',
        f'{profit_factor:.2f}',
        f'${avg_win:,.2f}',
        f'${avg_loss:,.2f}',
        '',
        f'${exit_pnls.max():,.2f}' if len(exit_pnls) > 0 else 'N/A',
        f'${exit_pnls.min():,.2f}' if len(exit_pnls) > 0 else 'N/A',
        f'${trades_df["cash_flow"].sum():,.2f}'
    ]
})

print("\n" + "=" * 60)
print("COMPREHENSIVE PERFORMANCE SUMMARY")
print("=" * 60)
print(summary.to_string(index=False))
print("=" * 60)

## Conclusion

This notebook demonstrates the **proper use of the backtester framework** for implementing an enhanced ATM straddle strategy.

### Framework Components Used:

1. **`MarketData` & `MarketDataLoader`**: Data management and vol surface construction
2. **`BacktestEngine` & `BacktestConfig`**: Core backtesting orchestration (extended for custom exits)
3. **`StraddleStrategy`**: Framework's built-in straddle position creation
4. **`Portfolio`**: Position tracking, trade history, and cash management
5. **`BlackScholesModel`**: Option pricing with market IV
6. **`PerformanceMetrics`**: Sharpe, Sortino, drawdown, and other analytics
7. **`VisualizationEngine`**: Equity curves, drawdown, returns, and Greeks charts

### Key Design:

- Extended `BacktestEngine` to add profit target/stop loss logic
- All position creation uses framework's `StraddleStrategy`
- Portfolio management via framework's `Portfolio` class
- Analytics via framework's `PerformanceMetrics`
- Visualizations via framework's `VisualizationEngine`

This approach **maximizes code reuse** and demonstrates proper framework integration.